In [ ]:
#importing Required Libraries
import torchvision
import torch.nn as nn
import torch

import torch.nn.functional as F
from torchvision import transforms,models,datasets
from torch import optim

import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import cv2, numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

!pip install torchsummary
import os
from glob import glob

In [ ]:
# upload our Kaggle authentication file
! pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# download the cats and dogs dataset and unzip it
! kaggle datasets download cashutosh/gender-classification-dataset

In [ ]:
! unzip gender-classification-dataset.zip

In [ ]:
train_data_dir = "/content/Training"
test_data_dir = "/content/Validation"

In [ ]:
transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(), # data augmentation
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # normalization
])

transforms_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dir = '/content/Training'
val_dir = '/content/Validation'

train_datasets = (datasets.ImageFolder(os.path.join(train_dir), transforms_train))
val_datasets = (datasets.ImageFolder(os.path.join(val_dir), transforms_val))

In [ ]:
print(type(train_datasets))

In [ ]:
# creates a DataLoader with a batch_size of 16
def get_data():

  train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=16, shuffle=True, num_workers=4)
  val_dataloader = torch.utils.data.DataLoader(val_datasets,  batch_size=16, shuffle=True, num_workers=4)

  return train_dataloader, val_dataloader

In [ ]:
data = train_datasets
im, label = data[50]

In [ ]:
plt.imshow(im.permute(1,2,0).cpu())
print(label)

In [ ]:
def get_model():

    model = models.vgg16(pretrained=True)
    for param in model.parameters():
        param.requires_grad = False

    model.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
    model.classifier = nn.Sequential(nn.Flatten(),

    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(128, 1),
    nn.Sigmoid())

    loss_fn = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr= 1e-3)

    return model.to(device), loss_fn, optimizer

In [ ]:
from torchsummary import summary

model, loss_fn, optimizer = get_model()
summary(model, input_size=(3, 224, 224))

In [ ]:
# Define the function that will train the model on a batch of data
def train_batch(x, y, model, opt, loss_fn):

    prediction = model(x)
    batch_loss = loss_fn(prediction, y)
    batch_loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    return batch_loss.item()

# Define the functions for calculating accuracy and validation loss
@torch.no_grad()
def accuracy(x, y, model):

    prediction = model(x)
    is_correct = (prediction > 0.5) == y
    return is_correct.cpu().numpy().tolist()

In [ ]:
@torch.no_grad()
def val_loss(x, y, model):
    prediction = model(x)
    val_loss = loss_fn(prediction, y)
    return val_loss.item()

In [ ]:
trn_dl, val_dl = get_data()
model, loss_fn, optimizer = get_model()

In [ ]:
# Train the model for 5 epochs and check the accuracy of the test data
train_accuracies = []
val_accuracies = []

for epoch in range(2):

    print(epoch)
    train_epoch_losses, train_epoch_accuracies = [], []
    val_epoch_accuracies = []

    for ix, batch in enumerate(iter(trn_dl)):
        #print(ix)
        x, y = batch
        x, y = x.float(), y.float().view(-1, 1)
        batch_loss = train_batch(x, y, model, optimizer, loss_fn)
        train_epoch_losses.append(batch_loss)

    for ix, batch in enumerate(iter(trn_dl)):
        x, y = batch
        is_correct = accuracy(x, y, model)
        train_epoch_accuracies.extend(is_correct)
    train_epoch_accuracy = np.mean(train_epoch_accuracies)

    for ix, batch in enumerate(iter(val_dl)):
        x, y = batch
        val_is_correct = accuracy(x, y, model)
        val_epoch_accuracies.extend(val_is_correct)

    val_epoch_accuracy = np.mean(val_epoch_accuracies)

    train_accuracies.append(train_epoch_accuracy)
    val_accuracies.append(val_epoch_accuracy)

In [ ]:
# plot the variation of the training and validation accuracies over increasing epochs
epochs = np.arange(2)+1

import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(epochs, train_accuracies, label='Training accuracy')
plt.plot(epochs, val_accuracies, label='Validation accuracy')

plt.title('Training and validation accuracy with 4K data points used for training')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
#plt.ylim(0.8,1)

plt.show()